In [1]:
import pandas as pl

In [2]:
df = pl.read_csv(r'data_example\data_Q1_2023\2023-01-01.csv')

In [3]:
df.head()

,date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2023-01-01,PL1331LAGDJ5GH,HGST HDS5C4040ALE630,4000787030016,0,100.0,0.0,134.0,101.0,139.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-01,PL2331LAGLW5UJ,HGST HMS5C4040BLE640,4000787030016,0,100.0,0.0,133.0,104.0,127.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-01,PL2331LAGM410J,HGST HMS5C4040BLE640,4000787030016,0,100.0,0.0,134.0,102.0,124.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-01,PL2331LAGM5NTJ,HGST HMS5C4040BLE640,4000787030016,0,100.0,0.0,135.0,99.0,121.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-01,PL2331LAGM5R4J,HGST HMS5C4040BLE640,4000787030016,0,100.0,0.0,135.0,99.0,123.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
import dask.dataframe as dd
import os
import glob

# Путь к папке с квартальными данными 
folder_path = "data_example/data_Q1_2023"

# Найдем все CSV файлы
csv_files = glob.glob(os.path.join(folder_path, '**', '*.csv'), recursive=True)

# Инициализируем пустой DataFrame для хранения результата
combined_df = None

# Чтение и объединение файлов
for csv_file in csv_files:
    print(f"Загрузка файла: {csv_file}")

    # Загрузим текущий CSV файл в Dask DataFrame
    df_temp = dd.read_csv(csv_file)
    
    if combined_df is None:
        # Если это первый файл, просто присваиваем его
        combined_df = df_temp
    else:
        # Выполним объединение по индексу используя outer join
        combined_df = dd.conca(combined_df, df_temp, how='outer', on='common_key', suffixes=('', '_dup'))

# Выполним вычисления, если использовали Dask DataFrame
combined_df = combined_df.compute()

# Сохраним как CSV файл
output_file = 'combined_data.csv'
combined_df.to_csv(output_file, index=False)

print(f'Данные сохранены в файл: {output_file}')


Загрузка файла: data_example/data_Q1_2023\2023-01-01.csv
Загрузка файла: data_example/data_Q1_2023\2023-01-02.csv


KeyError: 'common_key'

In [2]:
import polars as pl
import os
import dask.dataframe as dd

# Путь к папке с данными
folder_path = "data_example/data_Q1_2023"

# Используем Dask для чтения CSV файлов
ddf = dd.read_csv(os.path.join(folder_path, '*.csv'), assume_missing=True)

# Преобразуем Dask DataFrame в Pandas DataFrame
pandas_df = ddf.compute()

# Преобразуем Pandas DataFrame в Polars DataFrame
polars_df = pl.from_pandas(pandas_df)

# Если вам нужно объединение нескольких DataFrame, используйте pl.concat
# Для коункатенации добавить в список
combined_df = pl.concat([polars_df], how='diagonal_relaxed')


MemoryError: Unable to allocate 27.8 GiB for an array with shape (174, 21454992) and data type float64

In [5]:
import dask.dataframe as dd
import os
import glob

# Папка с данными (замените на свой путь)
folder_path = 'data_example/data_Q1_2023'

# Получаем список всех CSV файлов в папке
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# Баatch size - сколько файлов будем обрабатывать за раз (например, 10 файлов)
batch_size = 10

# Имена для выходных файлов (будем сохранять их частями)
output_file = 'combined_data.csv'

# Счетчик батчей для обработки по группам файлов
start_idx = 0

# Читаем и обрабатываем файлы по батчам
for batch_start in range(0, len(csv_files), batch_size):
    batch_files = csv_files[batch_start:batch_start + batch_size]
    print(f"Обработка файлов: {batch_files}")
    
    # Чтение файлов из данного батча
    dfs = [dd.read_csv(file, assume_missing=True) for file in batch_files]
    
    # Конкатенируем батч файлов по вертикали
    combined_batch_df = dd.concat(dfs, axis=0, interleave_partitions=True)
    
    # Выполняем вычисления и сохраняем в CSV
    mode = 'w' if batch_start == 0 else 'a'  # В первый раз "w", потом добавляем "a"
    header = True if batch_start == 0 else False  # Заголовок нужен только в первом батче
    
    combined_batch_df.compute().to_csv(output_file, index=False, mode=mode, header=header)

    print(f'Батч файлов {batch_files} успешно обработан и добавлен в {output_file}')

print(f'Все файлы объединены и сохранены в {output_file}')


Обработка файлов: ['data_example/data_Q1_2023\\2023-01-01.csv', 'data_example/data_Q1_2023\\2023-01-02.csv', 'data_example/data_Q1_2023\\2023-01-03.csv', 'data_example/data_Q1_2023\\2023-01-04.csv', 'data_example/data_Q1_2023\\2023-01-05.csv', 'data_example/data_Q1_2023\\2023-01-06.csv', 'data_example/data_Q1_2023\\2023-01-07.csv', 'data_example/data_Q1_2023\\2023-01-08.csv', 'data_example/data_Q1_2023\\2023-01-09.csv', 'data_example/data_Q1_2023\\2023-01-10.csv']
Батч файлов ['data_example/data_Q1_2023\\2023-01-01.csv', 'data_example/data_Q1_2023\\2023-01-02.csv', 'data_example/data_Q1_2023\\2023-01-03.csv', 'data_example/data_Q1_2023\\2023-01-04.csv', 'data_example/data_Q1_2023\\2023-01-05.csv', 'data_example/data_Q1_2023\\2023-01-06.csv', 'data_example/data_Q1_2023\\2023-01-07.csv', 'data_example/data_Q1_2023\\2023-01-08.csv', 'data_example/data_Q1_2023\\2023-01-09.csv', 'data_example/data_Q1_2023\\2023-01-10.csv'] успешно обработан и добавлен в combined_data.csv
Обработка файлов: [

In [7]:
pl.read_csv('combined_data.csv')

: 

In [2]:
import polars as pl

In [6]:
df = pl.read_csv_batched('combined_data.csv', batch_size=200)
df.head()

AttributeError: 'BatchedCsvReader' object has no attribute 'head'

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('combined_data.csv')

MemoryError: Unable to allocate 28.1 GiB for an array with shape (176, 21454992) and data type float64

In [19]:
df = pd.read_csv(r'data_example\data_Q1_2023\2023-01-02.csv')
df

,date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2023-01-02,PL1331LAGDJ5GH,HGST HDS5C4040ALE630,4000787030016,0,100.0,0.0,134.0,101.0,139.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-02,PL2331LAGLW5UJ,HGST HMS5C4040BLE640,4000787030016,0,100.0,0.0,133.0,104.0,127.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-02,PL2331LAGM410J,HGST HMS5C4040BLE640,4000787030016,0,100.0,0.0,134.0,102.0,124.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-02,PL2331LAGM5NTJ,HGST HMS5C4040BLE640,4000787030016,0,100.0,0.0,135.0,99.0,121.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-02,PL2331LAGM5R4J,HGST HMS5C4040BLE640,4000787030016,0,100.0,0.0,135.0,99.0,123.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235584,2023-01-02,3WJRXE8K,WDC WUH721816ALE6L4,16000900661248,0,100.0,0.0,136.0,96.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235585,2023-01-02,3WJXK5YK,WDC WUH721816ALE6L4,16000900661248,0,100.0,0.0,136.0,96.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235586,2023-01-02,3WJZY4EJ,WDC WUH721816ALE6L4,16000900661248,0,100.0,0.0,136.0,96.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235587,2023-01-02,3WK6NENK,WDC WUH721816ALE6L4,16000900661248,0,100.0,0.0,136.0,96.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df.columns

['date',
 'serial_number',
 'model',
 'capacity_bytes',
 'failure',
 'smart_1_normalized',
 'smart_1_raw',
 'smart_2_normalized',
 'smart_2_raw',
 'smart_3_normalized',
 'smart_3_raw',
 'smart_4_normalized',
 'smart_4_raw',
 'smart_5_normalized',
 'smart_5_raw',
 'smart_7_normalized',
 'smart_7_raw',
 'smart_8_normalized',
 'smart_8_raw',
 'smart_9_normalized',
 'smart_9_raw',
 'smart_10_normalized',
 'smart_10_raw',
 'smart_11_normalized',
 'smart_11_raw',
 'smart_12_normalized',
 'smart_12_raw',
 'smart_13_normalized',
 'smart_13_raw',
 'smart_15_normalized',
 'smart_15_raw',
 'smart_16_normalized',
 'smart_16_raw',
 'smart_17_normalized',
 'smart_17_raw',
 'smart_18_normalized',
 'smart_18_raw',
 'smart_22_normalized',
 'smart_22_raw',
 'smart_23_normalized',
 'smart_23_raw',
 'smart_24_normalized',
 'smart_24_raw',
 'smart_160_normalized',
 'smart_160_raw',
 'smart_161_normalized',
 'smart_161_raw',
 'smart_163_normalized',
 'smart_163_raw',
 'smart_164_normalized',
 'smart_164_raw

In [12]:
import os
import pandas as pd
import polars as pl

In [41]:
df = pd.read_csv(r'data_example\data_Q1_2023\2023-01-01.csv')

In [72]:
import os
import pandas as pd
import numpy as np

def update_smart_features(new_df, output_df):
    # Префиксы для SMART атрибутов
    smart_prefixes = ['smart_{}_normalized'.format(i) for i in range(1, 256)] + \
                     ['smart_{}_raw'.format(i) for i in range(1, 256)]
    
    # Убедимся, что у нас индексы актуальны для обоих DataFrame
    if 'serial_number' in new_df.columns:
        new_df.set_index('serial_number', inplace=True)
    
    # Проверим, что все нужные колонки из smart_prefixes присутствуют в new_df
    missing_cols_in_new_df = [col for col in smart_prefixes if col not in new_df.columns]
    # if missing_cols_in_new_df:
    #     print(f"Пропущены колонки в new_df: {missing_cols_in_new_df}")
    
    # 1. Определим недостающие колонки, которых нет в output_df
    missing_features = [col for col in smart_prefixes if col not in output_df.columns]

    # Если есть пропущенные колонки в output_df, создаем их с NaN
    if missing_features:
        missing_data = pd.DataFrame(np.nan, index=output_df.index, columns=missing_features)
        output_df = pd.concat([output_df, missing_data], axis=1)

    # 2. Определяем пересекающиеся индексы (serial numbers)
    common_serials = output_df.index.intersection(new_df.index)

    # Обновляем данные только для пересекающихся индексов и колонок, которые существуют в обоих DataFrame
    matching_columns = [col for col in smart_prefixes if col in new_df.columns and col in output_df.columns]

    if len(common_serials) > 0 and len(matching_columns) > 0:
        # Выполняем обновление для пересечений
        output_df.loc[common_serials, matching_columns] = new_df.loc[common_serials, matching_columns]
    
    # 3. Добавим новые строки из new_df, которых еще нет в output_df
    missing_serials = new_df.index.difference(output_df.index)

    if not missing_serials.empty:
        # Берем строки для отсутствующих serial_number
        new_data_to_add = new_df.loc[missing_serials, smart_prefixes]
        output_df = pd.concat([output_df, new_data_to_add])

    return output_df

# Функция для загрузки output.csv
def load_existing_output(output_path):
    if os.path.exists(output_path):
        # Загружаем уже существующие данные из CSV
        existing_data = pd.read_csv(output_path)

        # Устанавливаем serial_number как индекс, если столбец существует
        if 'serial_number' in existing_data.columns:
            existing_data.set_index('serial_number', inplace=True)
    
    else:
        # Если файла нет, создаем пустой DataFrame с нужными колонками

        # Стандартные колонки
        base_columns = ['model', 'capacity_bytes', 'hard_live_cost']

        # SMART признаки
        smart_columns = ['smart_{}_normalized'.format(i) for i in range(1, 256)] + \
                        ['smart_{}_raw'.format(i) for i in range(1, 256)]
        
        # Полный список колонок
        columns = base_columns + smart_columns
        
        # Создаем пустой DataFrame с колонками
        existing_data = pd.DataFrame(columns=columns)
        existing_data.index.name = 'serial_number'  # Устанавливаем serial_number как индекс для единообразия

    return existing_data

# Путь к output файлу
output_path = 'bibaboba.csv'


def update_features(new_df, output_new_df, is_last_file=False):
    
    ### Оптимизация через векторные операции,

    # 1. Разделим новые строки на существующие и новые диски
    existing_disks = new_df[new_df['serial_number'].isin(output_new_df.index)]  # Существующие ряды
    new_disks = new_df[~new_df['serial_number'].isin(output_new_df.index)]  # Новые ряды

    # 2. Обновим "hard_live_cost" для существующих дисков, где `failure == 0`
    if not existing_disks.empty:
        # Увеличиваем `hard_live_cost` на 1 только для строк, где failure==0
        mask = existing_disks['failure'] == 0
        update_serials = existing_disks[mask]['serial_number']
        
        # Обновляем hard_live_cost только для записей, где failure == 0
        if not is_last_file:
            output_new_df.loc[update_serials, 'hard_live_cost'] += 1
        else:
            output_new_df.loc[update_serials, 'hard_live_cost'] += 2000

    # 3. Добавим новые строки для новых дисков
    if not new_disks.empty and not is_last_file:
        # Подготавливаем DataFrame для новых строк
        new_disks_to_add = new_disks.set_index('serial_number')[
            ['model', 'capacity_bytes']]  # устанавливаем нужные колонки для добавления
        new_disks_to_add['hard_live_cost'] = 1  # hard_live_cost инициализируем как 1

        # Конкатенация новых строк
        output_new_df = pd.concat([output_new_df, new_disks_to_add])

    return output_new_df

# Загрузка существующих данных (или создание пустого DataFrame)
output_new_df = load_existing_output(output_path)
folder_path = r'data_example\data_Q1_2023/'
for idx, csv_data in enumerate(os.listdir(folder_path)):
    print(csv_data)
    is_last_file = (idx == len(csv_files) - 1)
    if csv_data.endswith('csv'):
        new_df = pd.read_csv(folder_path + csv_data)
        output_new_df = update_features(new_df, output_new_df, is_last_file=is_last_file)
        output_new_df = update_smart_features(new_df, output_new_df)
    if idx == 30:
        output_new_df.to_csv('sota_data.csv')

output_new_df.to_csv('sota_data.csv')

Пропущены колонки в new_df: ['smart_6_normalized', 'smart_14_normalized', 'smart_19_normalized', 'smart_20_normalized', 'smart_21_normalized', 'smart_25_normalized', 'smart_26_normalized', 'smart_27_normalized', 'smart_28_normalized', 'smart_29_normalized', 'smart_30_normalized', 'smart_31_normalized', 'smart_32_normalized', 'smart_33_normalized', 'smart_34_normalized', 'smart_35_normalized', 'smart_36_normalized', 'smart_37_normalized', 'smart_38_normalized', 'smart_39_normalized', 'smart_40_normalized', 'smart_41_normalized', 'smart_42_normalized', 'smart_43_normalized', 'smart_44_normalized', 'smart_45_normalized', 'smart_46_normalized', 'smart_47_normalized', 'smart_48_normalized', 'smart_49_normalized', 'smart_50_normalized', 'smart_51_normalized', 'smart_52_normalized', 'smart_53_normalized', 'smart_54_normalized', 'smart_55_normalized', 'smart_56_normalized', 'smart_57_normalized', 'smart_58_normalized', 'smart_59_normalized', 'smart_60_normalized', 'smart_61_normalized', 'smart

In [66]:
def load_existing_output(output_path):
    if os.path.exists(output_path):
        # Загружаем уже существующие данные из CSV
        existing_data = pd.read_csv(output_path)

        # Устанавливаем serial_number как индекс, если столбец существует
        if 'serial_number' in existing_data.columns:
            existing_data.set_index('serial_number', inplace=True)
    
    else:
        # Если файла нет, создаем пустой DataFrame с нужными колонками

        # Стандартные колонки
        base_columns = ['model', 'capacity_bytes', 'hard_live_cost']

        # SMART признаки
        smart_columns = ['smart_{}_normalized'.format(i) for i in range(1, 256)] + \
                        ['smart_{}_raw'.format(i) for i in range(1, 256)]
        
        # Полный список колонок
        columns = base_columns + smart_columns
        
        # Создаем пустой DataFrame с колонками
        existing_data = pd.DataFrame(columns=columns)
        existing_data.index.name = 'serial_number'  # Устанавливаем serial_number как индекс для единообразия

    return existing_data
q  = load_existing_output('')
q

,model,capacity_bytes,hard_live_cost,smart_1_normalized,smart_2_normalized,smart_3_normalized,smart_4_normalized,smart_5_normalized,smart_6_normalized,smart_7_normalized,...,smart_246_raw,smart_247_raw,smart_248_raw,smart_249_raw,smart_250_raw,smart_251_raw,smart_252_raw,smart_253_raw,smart_254_raw,smart_255_raw
serial_number,,,,,,,,,,,,,,,,,,,,,


In [60]:
output_new_df.to_csv('output.csv')

In [59]:
output_new_df['hard_live_cost'].value_counts()

hard_live_cost
90    146200
89     87906
71      1243
48      1241
34      1241
       ...  
32        12
74        10
63         9
39         8
26         7
Name: count, Length: 90, dtype: int64

In [56]:
import os
import polars as pl
import numpy as np

# Функция для загрузки output.csv
def load_existing_output(output_path):
    if os.path.exists(output_path):
        # Загружаем уже существующие данные из CSV
        existing_data = pl.read_csv(output_path)

        # Устанавливаем serial_number как индекс, если столбец существует
        if 'serial_number' in existing_data.columns:
            existing_data = existing_data.set_sorted('serial_number')
    else:
        # Если файла нет, создаем пустой DataFrame с нужными колонками и правильными типами
        existing_data = pl.DataFrame({
            'serial_number': pl.Series([], dtype=pl.Utf8),  # пустой столбец с типом строка
            'model': pl.Series([], dtype=pl.Utf8),
            'capacity_bytes': pl.Series([], dtype=pl.Int64),
            'hard_live_cost': pl.Series([], dtype=pl.Int64)
        })
    
    return existing_data

# Путь к output файлу
output_path = 'bibaboba.csv'

def update_features(new_df, output_new_df):
    ### Оптимизация через векторные операции

    # 1. Приводим типы данных к одинаковому формату (Int64)
    new_df = new_df.with_columns(
        new_df['capacity_bytes'].cast(pl.Int64)
    )
    
    # 2. Проверяем, есть ли столбец `hard_live_cost` в новом DataFrame. Если нет, создаем его.
    if 'hard_live_cost' not in new_df.columns:
        new_df = new_df.with_columns(pl.lit(0).alias('hard_live_cost'))

    # 3. Разделим новые строки на существующие и новые диски
    new_serials = new_df['serial_number'].to_numpy()
    existing_serials = output_new_df['serial_number'].to_numpy()

    # Векторное выделение существующих и новых дисков
    mask_existing = np.isin(new_serials, existing_serials)
    mask_new = ~mask_existing

    existing_disks = new_df.filter(mask_existing)
    new_disks = new_df.filter(mask_new)

    # 4. Обновим "hard_live_cost" для существующих дисков, где `failure == 0`
    if existing_disks.shape[0] > 0:
        # Маска для строк с failure == 0
        mask_failure = existing_disks['failure'].to_numpy() == 0
        update_serials = existing_disks['serial_number'].filter(mask_failure).to_numpy()

        # Увеличиваем hard_live_cost для этих serial_number
        mask_update = np.isin(existing_serials, update_serials)
        output_new_df = output_new_df.with_columns(
            pl.col('hard_live_cost').where(mask_update, pl.col('hard_live_cost') + 1)
        )

    # 5. Добавим новые строки для новых дисков
    if new_disks.shape[0] > 0:
        # Подготавливаем DataFrame для новых строк
        new_disks_to_add = new_disks.select(['serial_number', 'model', 'capacity_bytes'])
        new_disks_to_add = new_disks_to_add.with_columns(pl.lit(1).alias('hard_live_cost'))

        # Приводим типы столбцов к одинаковым типам
        new_disks_to_add = new_disks_to_add.with_columns(
            new_disks_to_add['capacity_bytes'].cast(pl.Int64),
            new_disks_to_add['hard_live_cost'].cast(pl.Int64)
        )

        # Конкатенация новых строк
        output_new_df = pl.concat([output_new_df, new_disks_to_add])

    return output_new_df

# Загрузка существующих данных (или создание пустого DataFrame)
output_new_df = load_existing_output(output_path)
folder_path = r'data_example\data_Q1_2023/'
for csv_data in os.listdir(folder_path):
    if csv_data.endswith('csv'):
        new_df = pl.read_csv(folder_path + csv_data)
        output_new_df = update_features(new_df, output_new_df)


KeyboardInterrupt: 

In [48]:
df

,date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2023-01-01,PL1331LAGDJ5GH,HGST HDS5C4040ALE630,4000787030016,0,100.0,0.0,134.0,101.0,139.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-01,PL2331LAGLW5UJ,HGST HMS5C4040BLE640,4000787030016,0,100.0,0.0,133.0,104.0,127.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-01,PL2331LAGM410J,HGST HMS5C4040BLE640,4000787030016,0,100.0,0.0,134.0,102.0,124.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-01,PL2331LAGM5NTJ,HGST HMS5C4040BLE640,4000787030016,0,100.0,0.0,135.0,99.0,121.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-01,PL2331LAGM5R4J,HGST HMS5C4040BLE640,4000787030016,0,100.0,0.0,135.0,99.0,123.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235592,2023-01-01,3WJRXE8K,WDC WUH721816ALE6L4,16000900661248,0,100.0,0.0,136.0,96.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235593,2023-01-01,3WJXK5YK,WDC WUH721816ALE6L4,16000900661248,0,100.0,0.0,136.0,96.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235594,2023-01-01,3WJZY4EJ,WDC WUH721816ALE6L4,16000900661248,0,100.0,0.0,136.0,96.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235595,2023-01-01,3WK6NENK,WDC WUH721816ALE6L4,16000900661248,0,100.0,0.0,136.0,96.0,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
output_df

,model,capacity_bytes,hard_live_cost
serial_number,,,
SN001,Model_A,1000,1
SN002,Model_B,2000,1
SN003,Model_C,3000,1
SN004,Model_D,4000,1
PL1331LAGDJ5GH,HGST HDS5C4040ALE630,4000787030016,1
...,...,...,...
3WJRXE8K,WDC WUH721816ALE6L4,16000900661248,1
3WJXK5YK,WDC WUH721816ALE6L4,16000900661248,1
3WJZY4EJ,WDC WUH721816ALE6L4,16000900661248,1


In [39]:
df.shape

(235597, 179)

In [38]:
output_df

,model,capacity_bytes,hard_live_cost
PL1331LAGDJ5GH,HGST HDS5C4040ALE630,4000787030016,1
PL2331LAGLW5UJ,HGST HMS5C4040BLE640,4000787030016,1
PL2331LAGM410J,HGST HMS5C4040BLE640,4000787030016,1
PL2331LAGM5NTJ,HGST HMS5C4040BLE640,4000787030016,1
PL2331LAGM5R4J,HGST HMS5C4040BLE640,4000787030016,1
...,...,...,...
PL2331LAH41XKJ,HGST HMS5C4040BLE640,4000787030016,1
PL2331LAH429EJ,HGST HMS5C4040BLE640,4000787030016,1
PL2331LAH437GJ,HGST HMS5C4040BLE640,4000787030016,1
PL2331LAH43HZJ,HGST HMS5C4040BLE640,4000787030016,1


In [ ]:
import os
import pandas as pd

# Путь к файлу с выводом результатов
output_path = 'output.csv'

# Функция для загрузки output.csv
def load_existing_output(output_path):
    if os.path.exists(output_path):
        # Загружаем уже существующие данные
        existing_data = pd.read_csv(output_path)
    else:
        # Если файла пока нет, создаем пустой DataFrame
        existing_data = pd.DataFrame(columns=['serial_number', 'hard_live_cost'])
    return existing_data

# Функция для обработки данных
def process_serial_numbers(data, existing_data):
    # Проверяем, что в исходных данных есть колонка Failure и номер Serial
    if 'failure' not in data.columns or 'serial_number' not in data.columns:
        raise ValueError("Данные не содержат необходимых столбцов 'failure' и 'SERIAL_NUMBER'.")
    
    # Итерация по состоянию дисков, в которых произошла ошибка
    for serial in data['SERIAL_NUMBER']:
        # Проверяем, есть ли этот SERIAL_NUMBER в existing_data
        if serial in existing_data['SERIAL_NUMBER'].values:
            # Увеличиваем счетчик `Cost_failure` на 1
            existing_data.loc[existing_data['SERIAL_NUMBER'] == serial, 'Cost_failure'] += 1
        else:
            # Добавляем новый SERIAL_NUMBER и инициализируем `Cost_failure` с 1
            new_record = {
                'SERIAL_NUMBER': serial,
                'Cost_failure': 1,
                'hard_live': None  # или инициализация значением по умолчанию, если требуется
            }
            existing_data = pd.concat([existing_data, pd.DataFrame([new_record])], ignore_index=True)
    
    return existing_data

# Функция для обновления output.csv после обработки данных
def update_output_file(existing_data, output_path):
    # Сохраняем обновленные данные обратно в output.csv
    existing_data.to_csv(output_path, index=False)

# главный процесс
def main():
    # Загружаем уже существующие данные из output.csv
    existing_data = load_existing_output(output_path)
    
    # Путь к папке с CSV-файлами
    data_directory = '/path_to_your_data_directory/'
    
    # Итерация по каждому кварталу и дню
    for quarter_folder in sorted(os.listdir(data_directory)):
        full_path = os.path.join(data_directory, quarter_folder)
        
        if os.path.isdir(full_path):
            for csv_file in sorted(os.listdir(full_path)):
                if csv_file.endswith('.csv'):
                    csv_path = os.path.join(full_path, csv_file)
                    
                    # Чтение данных на каждый день
                    day_data = pd.read_csv(csv_path)
                    
                    # Обработка данных для обновления output.csv
                    existing_data = process_serial_numbers(day_data, existing_data)
                    
                    # Обновляем output.csv после обработки очередного файла
                    update_output_file(existing_data, output_path)

if __name__ == "__main__":
    main()
